<a href="https://colab.research.google.com/github/lurking92/hw_sqj1/blob/main/%E3%80%8CBLIP2_%E9%96%8B%E7%99%BC%E3%80%8D_%E6%B8%AC%E8%A9%A6%E5%96%AE%E4%B8%80%E5%BD%B1%E7%89%87%EF%BC%8C%E8%AB%8B%E6%94%B9%E5%BD%B1%E7%89%87%E5%90%8D%E7%A8%B1(%E6%9C%89%E6%9B%B4%E6%96%B0%EF%BC%8C%E8%AB%8B%E6%8A%8A%E5%85%B6%E4%BB%96%E6%8F%9B%E6%8E%89).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile requirements.txt
opencc-python-reimplemented
torch>=2.0.0
transformers>=4.37.0
accelerate>=0.24.1
Pillow>=10.0.0
tqdm
opencv-python
ffmpeg-python
sentencepiece
sacremoses
mediapipe
scipy
numpy
Pillow
einops
segment_anything
requests

Overwriting requirements.txt


In [ ]:
# 安裝依賴
!pip install -r requirements.txt

In [ ]:
# 建立資料夾
!mkdir -p data/videos data/frames outputs scripts
!mkdir -p data/videos data/frames outputs scripts models

In [ ]:
import requests
import os

model_url = "https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task"
model_path = "models/pose_landmarker_heavy.task"

if not os.path.exists(model_path):
    print(f"下載 MediaPipe 姿態偵測模型到 {model_path}...")
    try:
        r = requests.get(model_url, allow_redirects=True)
        r.raise_for_status() # 檢查請求是否成功 (200 OK)
        with open(model_path, 'wb') as f:
            f.write(r.content)
        print("MediaPipe 姿態偵測模型下載完成。")
    except requests.exceptions.RequestException as e:
        print(f"下載模型時發生錯誤: {e}")
        print("請檢查您的網路連線或稍後再試。")
else:
    print("MediaPipe 姿態偵測模型已存在，跳過下載。")

MediaPipe 姿態偵測模型已存在，跳過下載。


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 上傳影片（可選）
from google.colab import files
uploaded = files.upload()
import shutil
import os
for fname in uploaded.keys():
    os.makedirs("data/videos", exist_ok=True)
    shutil.move(fname, f"data/videos/{fname}")

Saving my_test_video.mp4 to my_test_video.mp4


In [ ]:
%%writefile scripts/write_srt.py
def seconds_to_srt_time(seconds):
  hrs = seconds // 3600
  mins = (seconds % 3600) // 60
  secs = seconds % 60
  ms = int((seconds - int(seconds)) * 1000)
  return f"{int(hrs):02}:{int(mins):02}:{int(secs):02},{ms:03}"

def write_srt(captions, output_path):
  with open(output_path, 'w', encoding='utf-8') as f:
    for i, (start, end, text) in enumerate(captions, start=1):
      f.write(f"{i}\n")
      f.write(f"{seconds_to_srt_time(start)} --> {seconds_to_srt_time(end)}\n")
      f.write(f"{text}\n\n")

Overwriting scripts/write_srt.py


In [ ]:
%%writefile scripts/extract_frames.py
import os
import subprocess

def extract_frames(video_path, output_dir, interval_sec=2):
    os.makedirs(output_dir, exist_ok=True) # 確保輸出資料夾存在
    command = [
        "ffmpeg",
        "-i", video_path,
        "-vf", f"fps=1/{interval_sec}",
        os.path.join(output_dir, "frame_%03d.jpg") # 這裡會直接寫入 output_dir
    ]
    subprocess.run(command, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

Overwriting scripts/extract_frames.py


In [ ]:
%%writefile scripts/detect_objects.py
import torch
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection
from PIL import Image
import os
import io
import requests
from huggingface_hub import login # 保留 login，因為模型可能仍然需要，但它不會影響 CPU 運行

# 全局變數，用於載入模型一次
processor_gd = None
model_gd = None

# 使用最小的 Grounding DINO 模型
GROUNDING_DINO_MODEL_NAME = "IDEA-Research/grounding-dino-tiny"

def load_grounding_dino_model():
    """載入 Grounding DINO 模型。"""
    global processor_gd, model_gd
    if processor_gd is None or model_gd is None:
        print(f"載入 Grounding DINO 模型: {GROUNDING_DINO_MODEL_NAME}...")

        try:
            # 即使在 CPU 上運行，如果模型是門控的，Hugging Face token 仍然可能需要
            login(new_session=False)
            print("已檢測到 Hugging Face token (如果需要)。")
        except Exception:
            print("未檢測到 Hugging Face token 或登入失敗，但對於此模型可能不需要。")

        processor_gd = AutoProcessor.from_pretrained(GROUNDING_DINO_MODEL_NAME)
        model_gd = AutoModelForZeroShotObjectDetection.from_pretrained(GROUNDING_DINO_MODEL_NAME)

        # *** 恢復為自動判斷 GPU/CPU ***
        device = "cuda" if torch.cuda.is_available() else "cpu" # <--- 這裡改回來
        model_gd.to(device)
        print("Grounding DINO 模型載入完成。")
        print(f"Grounding DINO 模型運行設備: {next(model_gd.parameters()).device}")


def detect_objects(image_path, text_prompt, box_threshold=0.3, text_threshold=0.25):
    """
    使用 Grounding DINO 偵測圖片中的物體。

    Args:
        image_path (str): 圖片檔案的路徑。
        text_prompt (str): 用於偵測的文字提示 (例如: "person. dog. chair.").
        box_threshold (float): 邊界框的置信度閾值。
        text_threshold (float): 文字提示與偵測到的物體相關性的閾值。

    Returns:
        tuple: 包含偵測到的物體標籤列表和對應的邊界框列表。
               例如: (['person', 'chair'], [[x1,y1,x2,y2], [x1,y1,x2,y2]])
    """
    load_grounding_dino_model() # 確保模型已載入

    image = Image.open(image_path).convert("RGB")

    inputs = processor_gd(images=image, text=text_prompt, return_tensors="pt")

    # *** 恢復為將輸入資料移至模型所在的設備 ***
    inputs = {k: v.to(model_gd.device) for k, v in inputs.items()} # <--- 這裡改回來

    with torch.no_grad():
        outputs = model_gd(**inputs)

    results = processor_gd.post_process_grounded_object_detection(
        outputs,
        threshold=box_threshold,
        target_sizes=[image.size[::-1]]
    )[0]

    detected_labels = []
    detected_boxes = []

    boxes = results["boxes"].tolist()
    scores = results["scores"].tolist()
    text_labels = results["text_labels"]

    for score, label_text, box in zip(scores, text_labels, boxes):
        if score > text_threshold:
            detected_labels.append(label_text)
            detected_boxes.append(box)

    return detected_labels, detected_boxes

Overwriting scripts/detect_objects.py


In [ ]:
%%writefile scripts/segment_objects.py
import torch
import numpy as np
import cv2
from segment_anything import SamPredictor, sam_model_registry
from PIL import Image
import os
import requests # 導入 requests 函式庫
from tqdm import tqdm # 導入 tqdm 顯示下載進度

# 全局變數，用於載入模型一次
sam_predictor = None
SAM_CHECKPOINT_PATH = "/content/sam_vit_h_4b8939.pth" # SAM 模型權重路徑
SAM_CHECKPOINT_URL = "https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth" # SAM 模型下載 URL

def download_sam_checkpoint():
    """下載 SAM 模型權重（如果不存在）。"""
    if not os.path.exists(SAM_CHECKPOINT_PATH):
        print(f"下載 SAM 模型權重到 {SAM_CHECKPOINT_PATH}...")
        try:
            # 使用 requests 函式庫下載檔案
            response = requests.get(SAM_CHECKPOINT_URL, stream=True)
            response.raise_for_status() # 檢查是否有 HTTP 錯誤
            total_size_in_bytes = int(response.headers.get('content-length', 0))
            block_size = 1024 # 1 KB

            progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)
            with open(SAM_CHECKPOINT_PATH, 'wb') as file:
                for data in response.iter_content(block_size):
                    progress_bar.update(len(data))
                    file.write(data)
            progress_bar.close()

            if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
                print("錯誤：下載可能不完整！")
            else:
                print("SAM 模型權重下載完成。")
        except requests.exceptions.RequestException as e:
            print(f"下載 SAM 模型時發生錯誤: {e}")
            print("請檢查網絡連接或嘗試手動下載檔案並上傳到 /content/")
            exit(1) # 下載失敗則終止程式

def load_sam_model():
    """載入 Segment Anything Model (SAM) 預測器。"""
    global sam_predictor
    if sam_predictor is None:
        download_sam_checkpoint() # 確保權重已下載
        print("載入 Segment Anything Model (SAM)...")
        # 確保模型檢查點檔案存在
        if not os.path.exists(SAM_CHECKPOINT_PATH):
            print(f"錯誤：SAM 模型檢查點檔案 {SAM_CHECKPOINT_PATH} 不存在。")
            exit(1) # 檔案不存在則終止程式

        sam_model = sam_model_registry["vit_h"](checkpoint=SAM_CHECKPOINT_PATH)
        if torch.cuda.is_available():
            sam_model.to(device="cuda")
        sam_predictor = SamPredictor(sam_model)
        print("SAM 模型載入完成。")

def segment_objects(image_path, boxes):
    """
    使用 Segment Anything Model (SAM) 對圖片中的物體進行分割。

    Args:
        image_path (str): 圖片檔案的路徑。
        boxes (list): Grounding DINO 偵測到的物體的邊界框列表，格式為 [x1, y1, x2, y2]。

    Returns:
        list: 包含每個分割物件的 PIL.Image 物件列表 (mask) 或原始圖像。
              為了簡化，目前回傳原始圖像與遮罩的組合。
    """
    if not boxes: # 如果沒有偵測到物體，則不需要分割
        return []

    load_sam_model() # 確保模型已載入

    image_pil = Image.open(image_path).convert("RGB")
    image_np = np.array(image_pil) # 將 PIL 圖片轉換為 NumPy 陣列

    sam_predictor.set_image(image_np)

    # 確保 input_boxes 是 torch.Tensor 且在正確的 device 上
    input_boxes = torch.tensor(boxes, device=sam_predictor.device)

    # 進行預測
    # 這裡可能需要處理多個邊界框，predict 接受 N, 4 的 tensor
    # masks, scores, logits = sam_predictor.predict(
    #     point_coords=None,
    #     point_labels=None,
    #     box=input_boxes,
    #     multimask_output=False,
    # )
    # for boxes, (masks, iou_pred, low_res_logits) in zip(input_boxes, sam_predictor.predict_boxes(input_boxes)):
    # SAM's predict method takes a batch of boxes
    # Ref: https://github.com/facebookresearch/segment-anything/blob/main/segment_anything/predictor.py#L90

    # 批次處理boxes，如果input_boxes有多個，predict可以直接處理
    masks, _, _ = sam_predictor.predict_torch(
        point_coords=None,
        point_labels=None,
        boxes=input_boxes, # 這裡使用 boxes 參數
        multimask_output=False,
    )
    # masks 的形狀會是 (num_boxes, 1, H, W)


    segmented_images = []
    # masks 是一個張量，其形狀為 (N, 1, H, W)，N 是偵測到的物體數量
    for mask_tensor in masks:
        # 將布林遮罩轉換為 uint8
        mask_uint8 = mask_tensor.squeeze().cpu().numpy().astype(np.uint8) * 255

        # 創建一個只包含被遮罩區域的圖像
        masked_image_np = np.zeros_like(image_np)
        # 應用遮罩
        masked_image_np[mask_tensor.squeeze().cpu().numpy()] = image_np[mask_tensor.squeeze().cpu().numpy()]

        segmented_images.append(Image.fromarray(masked_image_np))

    return segmented_images

Overwriting scripts/segment_objects.py


In [ ]:
# scripts/analyze_pose.py
%%writefile scripts/analyze_pose.py
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
import numpy as np
from PIL import Image

# 全局變數，用於載入模型一次
pose_detector = None

def load_pose_detector():
    """載入 MediaPipe 姿態偵測模型 (使用 MediaPipe Tasks API)。"""
    global pose_detector
    if pose_detector is None:
        print("載入 MediaPipe 姿態偵測模型: models/pose_landmarker_heavy.task...")
        # 確保模型檔案路徑正確
        base_options = python.BaseOptions(model_asset_path='models/pose_landmarker_heavy.task')
        options = vision.PoseLandmarkerOptions(base_options=base_options, output_segmentation_masks=False)
        pose_detector = vision.PoseLandmarker.create_from_options(options)
        print("MediaPipe 姿態偵測模型載入完成。")

def get_pose_label(landmarks):
    """
    根據關鍵點判斷大致的姿態。
    這個函數需要根據實際需求細化，目前提供簡單示例。
    注意：這裡的判斷邏輯是基於 NormalizedLandmark 的 y 座標 (0.0 到 1.0)。
    """
    if not landmarks:
        return "無法辨識"

    # 獲取部分關鍵點的 Y 座標
    # MediaPipe 關鍵點索引參考: https://developers.google.com/mediapipe/solutions/vision/pose_landmarker
    # 例如：NOSE (0), LEFT_KNEE (25), RIGHT_KNEE (26), LEFT_ANKLE (27), RIGHT_ANKLE (28)

    # 使用 try-except 確保關鍵點存在
    try:
        nose_y = landmarks[mp.solutions.pose.PoseLandmark.NOSE.value].y
        left_knee_y = landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y
        right_knee_y = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].y
        left_ankle_y = landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].y
        right_ankle_y = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].y
    except IndexError:
        # 如果關鍵點列表不完整，無法進行判斷
        return "無法辨識"

    avg_knee_y = (left_knee_y + right_knee_y) / 2
    avg_ankle_y = (left_ankle_y + right_ankle_y) / 2

    # 計算軀幹的垂直跨度 (從鼻子到腳踝)
    vertical_span = avg_ankle_y - nose_y

    # 基於關鍵點的Y座標進行姿態判斷（這裡的閾值需要根據實際數據微調）
    # Y軸向下為正，值越大表示位置越低

    # 跌倒：身體非常低，垂直跨度非常小
    # 假設圖像高度為1，如果鼻子y座標很高（接近底部）且垂直跨度小
    if nose_y > 0.65 and vertical_span < 0.2: # 鼻子靠近底部，且身體被壓縮
        return "跌倒"
    # 坐下：鼻子和膝蓋的Y座標差異小（膝蓋相對較高），且身體總體高度較低
    elif (nose_y - avg_knee_y) < 0.1 and vertical_span < 0.5: # 膝蓋靠近鼻子，且身體不完全伸展
        return "坐下"
    # 走路：軀幹垂直跨度在一定範圍內，表示處於中間姿態
    elif 0.5 <= vertical_span <= 0.8:
        return "走路"
    # 站立：軀幹垂直跨度較大，身體伸展
    elif vertical_span > 0.8:
        return "站立"
    else:
        return "活動" # 預設為活動，或表示無法精確分類


def analyze_pose(image_path): # *** 保持這個函數名稱為 analyze_pose ***
    """
    分析圖片中的人體姿態並返回一個標籤。
    """
    load_pose_detector() # 確保模型已載入

    # 載入圖片並轉換為 MediaPipe 圖像格式
    image = Image.open(image_path).convert("RGB")
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=np.asarray(image))

    # 執行姿態偵測
    detection_result = pose_detector.detect(mp_image)

    # 提取關鍵點
    if detection_result.pose_landmarks:
        # MediaPipe 可能偵測到多個人，這裡簡化處理，只取第一個人的姿態
        landmarks = detection_result.pose_landmarks[0] # 獲取第一個偵測到的人的關鍵點列表

        # 獲取姿態標籤
        pose_label = get_pose_label(landmarks)
        return pose_label
    else:
        return "無法辨識"

Overwriting scripts/analyze_pose.py


In [ ]:
%%writefile scripts/utils.py
import subprocess
import json
import datetime # 需要 datetime 模組來處理時間
from datetime import timedelta

def get_video_duration_ffmpeg(video_path):
    """
    使用 ffmpeg 獲取影片的時長 (秒)。
    """
    try:
        command = [
            'ffprobe',
            '-v', 'error',
            '-show_entries', 'format=duration',
            '-of', 'default=noprint_wrappers=1:nokey=1',
            video_path
        ]
        duration_str = subprocess.check_output(command, stderr=subprocess.STDOUT).decode().strip()
        return float(duration_str)
    except Exception as e:
        print(f"Error getting video duration with ffmpeg: {e}")
        return None

def save_txt(captions_list, output_path):
    """
    將字幕列表保存為純文字檔案 (.txt)。
    captions_list 預期是一個只有字幕文字的列表。
    """
    with open(output_path, 'w', encoding='utf-8') as f:
        for caption in captions_list:
            f.write(caption + '\n')
    print(f"字幕已儲存為純文字檔案到 {output_path}")

# 注意：save_transcription_to_srt 函數已移除，因為 main.py 不再需要它。

Overwriting scripts/utils.py


In [ ]:
%%writefile scripts/generate_caption.py
# scripts/generate_caption.py
from transformers import Blip2Processor, Blip2ForConditionalGeneration, pipeline
from PIL import Image
import torch
import re
import os
from opencc import OpenCC

# 模型設定
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16 if device == "cuda" else torch.float32)
model = model.to(device)
print(f"BLIP2 模型運行設備: {next(model.parameters()).device}")

# 翻譯和轉換
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-zh", src_lang="en", tgt_lang="zh", device=0 if torch.cuda.is_available() else -1)
cc = OpenCC('s2twp')

# 無效描述的過濾清單 (新增了更多通用詞和網址相關詞)
INVALID_DESCRIPTIONS = [
    "gta san andreas", "screenshot", "jpg", "png", "image", "photo", "picture",
    "describe", "what", "you", "see", "in", "the", "image", "this", "is", "a",
    "man", "woman", "person", "standing", "sitting", "walking", "running",
    "game", "video", "frame", "scene", "view", "show", "display", "appear",
    "can", "will", "be", "given", "option", "mind", "words", "few", "left",
    "right", "front", "back", "name", "set", "have", "to", "your",
    "youtube", "youtu", "com", "http", "https", "www", "googleusercontent" # 新增網址相關詞
]

def is_valid_description(text):
    """檢查描述是否有效"""
    if not text or len(text.strip()) < 5:
        return False

    # 轉換為小寫並移除標點符號
    clean_text_lower = re.sub(r'[^\w\s]', '', text.lower())
    words = clean_text_lower.split()

    # 檢查是否包含太多無效詞彙
    invalid_count = sum(1 for word in words if word in INVALID_DESCRIPTIONS)
    if len(words) > 0 and invalid_count / len(words) > 0.6:  # 如果超過60%是無效詞彙
        return False

    # 檢查是否是常見的無效模式 (新增網址模式)
    invalid_patterns = [
        r"^describe.*image", r"^what.*see", r"^you.*describe",
        r"^this.*screenshot", r"^gta.*andreas", r"^the.*game",
        r"^a\s+(man|woman|person)\s+is\s+$", r"^.*standing.*front.*$",
        r"https?://(?:www\.)?(?:youtube\.com|youtu\.be|googleusercontent\.com/youtube\.com)/?\S*", # 匹配各種 YouTube 網址
        r"youtube", r"youtu", r"com", r"http", r"https", r"www", r"googleusercontent" # 再次檢查關鍵詞
    ]

    for pattern in invalid_patterns:
        if re.search(pattern, clean_text_lower): # 對清理過的小寫文本進行匹配
            return False

    return True

def clean_text(text):
    """清理生成的文字，強化去重複和網址移除"""
    if not text:
        return ""

    # 移除常見的無用前綴和後綴 (包含中文和英文)
    text = re.sub(r"^(this is |there is |the image shows |image shows |a man |a woman |a person |screenshot of |image of )", "", text, flags=re.IGNORECASE)
    text = re.sub(r"^(這是|影像顯示|圖片顯示|畫面顯示|一個人|一個男人|一個女人|螢幕截圖)", "", text)

    # 移除無意義的結尾
    text = re.sub(r"(\s+in\s+the\s+image|\s+of\s+the\s+game|\s+in\s+gta|\s+screenshot)$", "", text, flags=re.IGNORECASE)

    # 移除各種 YouTube 網址和相關關鍵字
    text = re.sub(r"https?://(?:www\.)?(?:youtube\.com|youtu\.be|googleusercontent\.com/youtube\.com)/?\S*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"(youtube|youtu|com|http|https|www|googleusercontent)\s*", "", text, flags=re_IGNORECASE) # 移除關鍵詞

    # 移除重複的詞語：例如 "椅子 椅子" -> "椅子", "表格表格表格" -> "表格"
    # 這段正則表達式會匹配連續重複的中文詞或英文單詞，並替換為單個
    text = re.sub(r"([\u4e00-\u9fa5a-zA-Z0-9]+)\s*\1+", r"\1", text) # 處理 "詞 詞"
    text = re.sub(r"([\u4e00-\u9fa5a-zA-Z0-9]{2,})\1+", r"\1", text) # 處理 "詞詞詞" (至少兩個字重複)

    # 移除多餘的標點和空格
    text = re.sub(r"\s+", " ", text)
    text = text.strip(" .,。，")

    return text

def generate_better_caption(image_path, pose_label=None, detected_objects=None):
    """為單張圖像生成更好的描述"""
    image = Image.open(image_path).convert("RGB")

    # 嘗試多種生成策略
    best_caption = ""
    attempts = 0
    max_attempts = 3

    while attempts < max_attempts and not best_caption:
        try:
            prompt = "Describe this image." # 統一使用簡潔提示

            inputs = processor(images=image, text=prompt, return_tensors="pt").to(model.device)

            with torch.no_grad():
                generated_ids = model.generate(
                    **inputs,
                    max_new_tokens=30,
                    min_length=5,
                    num_beams=5,
                    early_stopping=True,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                    repetition_penalty=1.2,
                    length_penalty=1.0,
                    no_repeat_ngram_size=2
                )

            generated_text = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)

            # 移除提示詞部分
            # 這裡需要更精確地移除輸入的 prompt 部分
            # 找到 prompt 在 generated_text 中的位置，然後移除
            if prompt in generated_text:
                generated_text = generated_text.replace(prompt, "").strip()

            # 檢查生成的文字是否有效
            if is_valid_description(generated_text):
                best_caption = generated_text
                break

        except Exception as e:
            print(f"生成嘗試 {attempts + 1} 失敗: {e}")

        attempts += 1

    # 如果所有嘗試都失敗，使用後備方案
    if not best_caption:
        if pose_label and pose_label != "無法辨識":
            best_caption = f"person {pose_label}"
        elif detected_objects:
            # 嘗試從物件中組合一個簡單描述
            filtered_objects = [obj for obj in detected_objects if obj.lower() != "person"]
            if filtered_objects:
                best_caption = f"scene with {', '.join(filtered_objects[:2])}" # 最多兩個物件
            else:
                best_caption = "scene"
        else:
            best_caption = "scene" # 最簡單的後備

    return best_caption

def generate_caption(image_paths_batch, pose_labels_batch=None, detected_objects_batch=None, debug_log_path="outputs/debug.log"):
    """生成圖像描述字幕 - 改進版"""
    debug_log = open(debug_log_path, "a", encoding="utf-8")

    # 為每張圖像生成描述
    raw_english_captions = []
    for idx, image_path in enumerate(image_paths_batch):
        pose_label = pose_labels_batch[idx] if pose_labels_batch else None
        detected_objects = detected_objects_batch[idx] if detected_objects_batch else None

        caption = generate_better_caption(image_path, pose_label, detected_objects)
        raw_english_captions.append(caption)

    print(f"英文描述: {raw_english_captions}")

    # 翻譯成中文
    translated_texts = []
    for caption in raw_english_captions:
        if caption.strip():
            try:
                translated = translator(caption)
                if isinstance(translated, list) and len(translated) > 0:
                    if isinstance(translated[0], dict) and 'translation_text' in translated[0]:
                        translated_texts.append(translated[0]['translation_text'])
                    else:
                        translated_texts.append(str(translated[0]))
                else:
                    translated_texts.append("")
            except Exception as e:
                print(f"翻譯失敗: {e}")
                translated_texts.append("")
        else:
            translated_texts.append("")

    # 生成最終字幕
    final_zh_captions = []
    for idx, (raw_caption, zh_translation) in enumerate(zip(raw_english_captions, translated_texts)):
        # 轉換成繁體中文並清理
        zh_trad = cc.convert(zh_translation) if zh_translation else ""
        clean_caption = clean_text(zh_trad) # 對翻譯後的文字進行清理

        # 構建基礎描述
        base_description = clean_caption if clean_caption else "畫面內容" # 如果清理後為空，使用通用詞

        # 添加姿勢信息
        pose_info = ""
        if pose_labels_batch and pose_labels_batch[idx] and pose_labels_batch[idx] != "無法辨識":
            # 避免與基礎描述重複，例如如果基礎描述已經是「一個人走路」，就不再重複「走路」
            if pose_labels_batch[idx] not in base_description:
                pose_info = f"，{pose_labels_batch[idx]}"
            else:
                pose_info = f"（{pose_labels_batch[idx]}）" # 動作標籤，用括號表示補充

        # 添加物件信息
        object_info = ""
        if detected_objects_batch and detected_objects_batch[idx]:
            unique_objects = list(set(detected_objects_batch[idx]))
            filtered_objects = [obj for obj in unique_objects if obj.lower() != "person"]

            if filtered_objects:
                try:
                    # 只翻譯前3個最重要的物件
                    objects_to_translate = filtered_objects[:3]
                    translated_objects_raw = translator(objects_to_translate)
                    zh_objects_cleaned = []

                    for obj_trans in translated_objects_raw:
                        translated_obj_text = ""
                        if isinstance(obj_trans, dict) and 'translation_text' in obj_trans:
                            translated_obj_text = obj_trans['translation_text']
                        elif isinstance(obj_trans, str):
                            translated_obj_text = obj_trans
                        elif isinstance(obj_trans, list) and len(obj_trans) > 0 and isinstance(obj_trans[0], dict) and 'translation_text' in obj_trans[0]:
                            translated_obj_text = obj_trans[0]['translation_text']

                        if translated_obj_text:
                            # 對單個物件的翻譯結果也進行清理和去重複
                            cleaned_obj_text = clean_text(cc.convert(translated_obj_text))
                            if cleaned_obj_text and cleaned_obj_text not in zh_objects_cleaned:  # 避免重複添加
                                zh_objects_cleaned.append(cleaned_obj_text)

                    if zh_objects_cleaned:
                        object_info = f"，可見{', '.join(zh_objects_cleaned)}"
                except Exception as e:
                    print(f"物件翻譯失敗: {e}")

        # 組合最終字幕
        final_caption = f"{base_description}{pose_info}{object_info}"

        # 如果最終字幕仍然包含過多無效詞或過短，則使用更通用的後備
        if not is_valid_description(final_caption) or len(final_caption.strip()) < 5:
            if pose_labels_batch and pose_labels_batch[idx] and pose_labels_batch[idx] != "無法辨識":
                final_caption = f"畫面中一個人正在{pose_labels_batch[idx]}"
            elif detected_objects_batch and detected_objects_batch[idx]:
                filtered_objects = [obj for obj in detected_objects_batch[idx] if obj.lower() != "person"]
                if filtered_objects:
                    # 再次嘗試翻譯和清理物件
                    try:
                        objects_to_translate = filtered_objects[:2]
                        translated_objects_raw = translator(objects_to_translate)
                        zh_objects_fallback = []
                        for obj_trans in translated_objects_raw:
                            translated_obj_text = ""
                            if isinstance(obj_trans, dict) and 'translation_text' in obj_trans:
                                translated_obj_text = obj_trans['translation_text']
                            elif isinstance(obj_trans, str):
                                translated_obj_text = obj_trans
                            elif isinstance(obj_trans, list) and len(obj_trans) > 0 and isinstance(obj_trans[0], dict) and 'translation_text' in obj_trans[0]:
                                translated_obj_text = obj_trans[0]['translation_text']
                            if translated_obj_text:
                                cleaned_obj_text = clean_text(cc.convert(translated_obj_text))
                                if cleaned_obj_text and cleaned_obj_text not in zh_objects_fallback:
                                    zh_objects_fallback.append(cleaned_obj_text)
                        if zh_objects_fallback:
                            final_caption = f"畫面中可見{', '.join(zh_objects_fallback)}"
                        else:
                            final_caption = "畫面內容不詳"
                    except Exception:
                        final_caption = "畫面內容不詳"
                else:
                    final_caption = "畫面內容不詳"
            else:
                final_caption = "畫面內容不詳"


        # 記錄調試信息
        debug_log.write(f"圖像: {image_paths_batch[idx]}\n")
        debug_log.write(f"  原始英文: {raw_caption}\n")
        debug_log.write(f"  中文翻譯: {zh_translation}\n")
        debug_log.write(f"  清理後: {clean_caption}\n")
        debug_log.write(f"  姿勢標籤: {pose_labels_batch[idx] if pose_labels_batch else 'None'}\n")
        debug_log.write(f"  偵測物件: {detected_objects_batch[idx] if detected_objects_batch else 'None'}\n")
        debug_log.write(f"  最終結果: {final_caption}\n\n")

        final_zh_captions.append(final_caption)

    debug_log.close()
    return final_zh_captions

Overwriting scripts/generate_caption.py


In [ ]:
%%writefile main.py
import os
import shutil
import time
from datetime import timedelta
from google.colab import drive
from tqdm.notebook import tqdm # 引入 tqdm

# 導入所有必要的腳本函數
from scripts.extract_frames import extract_frames
from scripts.detect_objects import detect_objects
from scripts.segment_objects import segment_objects # 即使目前未啟用，也需要確保能導入
from scripts.analyze_pose import analyze_pose
from scripts.generate_caption import generate_caption
from scripts.utils import save_txt, get_video_duration_ffmpeg # 只導入 TXT 和 FFmpeg 相關函數

# --- 設定變數 ---
VIDEO_FILE_NAME = "my_test_video.mp4" # 請替換為你的影片檔案名稱
INPUT_VIDEO_PATH = f"/content/drive/MyDrive/{VIDEO_FILE_NAME}" # 影片在 Google Drive 中的路徑
OUTPUT_DIR = "/content/drive/MyDrive/Colab_Subtitles_Output" # 所有輸出的根目錄
FRAMES_DIR = os.path.join(OUTPUT_DIR, "frames", os.path.splitext(VIDEO_FILE_NAME)[0]) # 影片幀的輸出目錄
TEMP_OBJECTS_DIR = os.path.join(OUTPUT_DIR, "temp_objects", os.path.splitext(VIDEO_FILE_NAME)[0]) # 分割物體的臨時目錄
# SRT_OUTPUT_PATH 不再需要
TXT_OUTPUT_PATH = os.path.join(OUTPUT_DIR, f"{os.path.splitext(VIDEO_FILE_NAME)[0]}.txt") # TXT 字幕輸出路徑

FRAME_INTERVAL = 2 # 每隔多少秒提取一幀

# 每批次處理多少幀 (根據你的 GPU 記憶體調整，越大越快但越耗記憶體)
BATCH_SIZE = 8 # 建議值，可以嘗試調整

# --- 確保輸出目錄存在 ---
os.makedirs(FRAMES_DIR, exist_ok=True)
os.makedirs(TEMP_OBJECTS_DIR, exist_ok=True) # 確保物體分割臨時目錄存在

# --- 步驟 0: 確認 Google Drive 已掛載 ---
print("--- 步驟 0: 確認 Google Drive 已掛載 ---")
if os.path.exists("/content/drive/MyDrive"):
    print("Google Drive 已掛載。")
else:
    print("警告: Google Drive 未掛載，請確保已運行相關掛載步驟。")

# --- 步驟 1: 提取影片幀 ---
print("\n--- 步驟 1: 提取影片幀 ---")
# 檢查是否已存在，如果不存在則提取
if not os.path.exists(os.path.join(FRAMES_DIR, "frame_00001.jpg")):
    print(f"[1] 正在提取影片 {VIDEO_FILE_NAME} 的圖片幀...")
    extract_frames(INPUT_VIDEO_PATH, FRAMES_DIR, FRAME_INTERVAL)
    print(f"[1] 影片幀已提取到: {FRAMES_DIR}")
else:
    print(f"[1] 影片 {VIDEO_FILE_NAME} 的圖片幀已存在於 Google Drive ({FRAMES_DIR})，跳過提取。")

# --- 步驟 2: 處理每一幀以生成字幕 ---
print(f"\n--- 步驟 2: 處理 {VIDEO_FILE_NAME} 的每一幀並生成字幕 ---")

# 讀取所有圖片幀的路徑，並按數字順序排序
frame_files = sorted([f for f in os.listdir(FRAMES_DIR) if f.endswith('.jpg')])
image_paths = [os.path.join(FRAMES_DIR, f) for f in frame_files]

# 初始化保存字幕的列表 (只包含字幕文字，因為不再需要SRT的時間戳)
all_transcriptions_text_only = []
existing_transcriptions_count = 0

# 檢查字幕 TXT 檔案是否存在，如果存在則讀取它並跳過已處理的幀
if os.path.exists(TXT_OUTPUT_PATH):
    print(f"警告：影片 {VIDEO_FILE_NAME} 的 TXT 檔案已存在，將嘗試從中恢復進度。")
    with open(TXT_OUTPUT_PATH, 'r', encoding='utf-8') as f:
        existing_lines = f.readlines()
        existing_transcriptions_count = len(existing_lines)
        for line in existing_lines:
            all_transcriptions_text_only.append(line.strip())

    print(f"已從 TXT 檔案中恢復 {existing_transcriptions_count} 條現有字幕。")
    # 如果已處理的幀數與總幀數相同，則直接完成
    if existing_transcriptions_count >= len(image_paths):
        print(f"所有幀都已在 TXT 檔案中處理過。")
        print("所有任務完成！你可以在 Google Drive 中找到 TXT 輸出。") # 統一成功提示
        exit() # 退出程式

# 從上次中斷的地方繼續處理
start_frame_index = existing_transcriptions_count
print(f"將從第 {start_frame_index + 1} 幀開始處理 (共 {len(image_paths)} 幀)。")

# 刪除上次可能殘留的臨時物體分割圖像 (每次運行都清理，避免積累)
if os.path.exists(TEMP_OBJECTS_DIR):
    shutil.rmtree(TEMP_OBJECTS_DIR)
os.makedirs(TEMP_OBJECTS_DIR, exist_ok=True)

# 使用 tqdm 顯示進度條
with tqdm(total=len(image_paths), initial=start_frame_index, desc=f"生成 {VIDEO_FILE_NAME} 的字幕") as pbar:
    for i in range(start_frame_index, len(image_paths), BATCH_SIZE):
        batch_image_paths = image_paths[i:i + BATCH_SIZE]

        # --- 每個批次處理前的時間戳 ---
        batch_start_time = time.time()

        # Step 2.1: 偵測物體
        print(f"\n[進度 {pbar.n}/{len(image_paths)}] 正在偵測批次物體 (幀 {i+1} 到 {min(i + BATCH_SIZE, len(image_paths))})...")
        batch_objects = []
        for img_path in batch_image_paths:
            objects, _ = detect_objects(img_path, text_prompt="person, car, animal, road, building, tree, sky, water, table, chair, computer, phone, book, cup, food, clothes, bag, hand, head, body, sitting, standing, walking, running, jumping, eating, drinking, holding, looking, talking, working")
            batch_objects.append(objects)
        print(f"[進度 {pbar.n}/{len(image_paths)}] 物體偵測完成。")

        # Step 2.2: 分析姿態
        print(f"[進度 {pbar.n}/{len(image_paths)}] 正在分析批次姿態...")
        batch_pose_labels = []
        for img_path in batch_image_paths:
            pose_label = analyze_pose(img_path)
            batch_pose_labels.append(pose_label)
        print(f"[進度 {pbar.n}/{len(image_paths)}] 姿態分析完成。")

        # Step 2.3: 生成字幕
        print(f"[進度 {pbar.n}/{len(image_paths)}] 正在生成批次字幕...")
        batch_captions = generate_caption(batch_image_paths, batch_pose_labels, batch_objects)
        print(f"[進度 {pbar.n}/{len(image_paths)}] 字幕生成完成。")

        # 將生成的字幕添加到總列表
        all_transcriptions_text_only.extend(batch_captions)

        # 每個批次處理後的總時間
        batch_end_time = time.time()
        batch_duration = batch_end_time - batch_start_time
        print(f"[進度 {pbar.n}/{len(image_paths)}] 批次處理時間: {timedelta(seconds=batch_duration)}")

        # 定期保存進度到 TXT 檔案
        save_txt(all_transcriptions_text_only, TXT_OUTPUT_PATH)
        print(f"[進度 {pbar.n}/{len(image_paths)}] 進度已保存到 TXT 檔案: {TXT_OUTPUT_PATH}")

        # 更新 tqdm 進度條
        pbar.update(len(batch_image_paths))


print("\n[3] 所有幀處理完成。")

# --- 步驟 3: 字幕處理完成，確認 TXT 檔案已保存 ---
print(f"\n--- 步驟 3: 字幕處理完成，確認 TXT 檔案已保存 ---")
print(f"最終 TXT 字幕已保存到: {TXT_OUTPUT_PATH}")

print("\n所有任務完成！你可以在 Google Drive 中找到 TXT 輸出。")

Overwriting main.py


In [ ]:
# 執行字幕生成（main.py）
!python main.py

2025-07-13 16:56:16.046302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752425776.236636   37379 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752425776.284848   37379 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-13 16:56:16.697795: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be t

In [ ]:
# 顯示字幕內容
from glob import glob
for srt_file in glob("outputs/*.srt"):
    print("\n=====", srt_file, "=====")
    with open(srt_file, encoding='utf-8') as f:
        print(f.read())